In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from os.path import join

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia

In [ ]:
saude = pd.read_excel(join(default_input, 'outros', 'saude.xlsx'))
saude.shape[0]

In [ ]:
# saude = saude.loc[saude['classificacao_final'] == 2]
saude = saude.loc[saude['uf_residencia'] == 41]

In [ ]:
saude = saude.loc[saude['estudante']=='Não']
saude = saude.loc[saude['parente']=='Não']
saude = saude.loc[saude['uf_residencia'] == 41]
saude = saude.loc[((saude['status_notificacao'] == 1) | (saude['status_notificacao'] == 2))]
saude = saude.loc[saude['excluir_ficha'] == 2]
saude.shape[0]

In [ ]:
saude.loc[saude['evolucao'] == 5, 'evolucao'] = 2
saude.loc[saude['cod_cbo'].isnull(), 'cod_cbo'] = 0
saude.loc[saude['cbo_descricao'].isnull(), 'cbo_descricao'] = 'Não Informado'

In [ ]:
saude.loc[saude['ocupacao_descricao'].isnull(), 'Área'] = 'Não Informado'

In [ ]:
saude.loc[saude['enfermagem']=='Sim', 'Área'] = 'Enfermagem'
saude.loc[saude['medico']=='Sim', 'Área'] = 'Médico'
saude.loc[saude['servicos']=='Sim', 'Área'] = 'Serviços'
saude.loc[saude['administrativo']=='Sim', 'Área'] = 'Administrativa'
saude.loc[saude['agente_saude']=='Sim', 'Área'] = 'Agente Comunitário de Saúde'
saude.loc[saude['farmaceutica']=='Sim', 'Área'] = 'Área Farmaceutica'
saude.loc[saude['transporte']=='Sim', 'Área'] = 'Transporte'
saude.loc[saude['odontologia']=='Sim', 'Área'] = 'Odontologia'
saude.loc[saude['fisioterapia']=='Sim', 'Área'] = 'Fisioterapia'
saude.loc[saude['laboratorial']=='Sim', 'Área'] = 'Área Laboratorial/Diagnóstico'
saude.loc[saude['alimentacao']=='Sim', 'Área'] = 'Alimentação'
saude.loc[saude['veterinaria']=='Sim', 'Área'] = 'Veterinária'
saude.loc[saude['socorrista']=='Sim', 'Área'] = 'Socorrista'
saude.loc[saude['servico_social']=='Sim', 'Área'] = 'Serviço Social'
saude.loc[saude['cuidador_idosos']=='Sim', 'Área'] = 'Cuidador de Idosos'
saude.loc[saude['fonoaudiologia']=='Sim', 'Área'] = 'Fonoaudiologia'
saude.loc[saude['psicologia']=='Sim', 'Área'] = 'Psicologia'

In [ ]:
saude.loc[saude['Área'].isnull(), 'Área'] = 'Outros Trabalhadores'

In [ ]:
saude.columns

In [ ]:
saude['cbo_n'] = saude['Área']

In [ ]:
teste = saude.drop(columns={'enfermagem', 'medico','servicos', 'administrativo', 'agente_saude', 'farmaceutica',
       'transporte', 'odontologia', 'fisioterapia', 'laboratorial',
       'alimentacao', 'veterinaria', 'socorrista', 'servico_social',
       'cuidador_idosos', 'fonoaudiologia', 'psicologia', 'estudante','Área',
       'parente'})

In [ ]:
teste.columns

In [ ]:
teste.to_excel(join(default_input, 'outros', 'saude_modificado.xlsx'))

In [ ]:
saude.to_excel('saude_casos.xlsx', index = False)

In [ ]:
#saude['data_liberacao'] = saude['data_liberacao'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors = 'coerce'))
#saude['data_notificacao'] = saude['data_notificacao'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors = 'coerce'))
#saude.loc[saude['data_liberacao'].isnull(), 'data_liberacao'] = saude.loc[saude['data_liberacao'].isnull(), 'data_notificacao']
#saude['mes_liberacao'] = pd.Series(saude['data_liberacao'].values.astype('datetime64[M]'))

In [ ]:
#tabela = saude.groupby(['Área', 'mes_liberacao'])[['paciente']].count().unstack()
#tabela.to_excel('TABELA_comparacao.xlsx')

In [ ]:
tabela_not = saude.groupby('Área')[['paciente']].count().rename(columns={'paciente':'Notificados'}).sort_values(by=['Notificados'], ascending = False).reset_index()
tabela_conf = saude.loc[saude['classificacao_final']==2].groupby('Área')[['paciente']].count().rename(columns={'paciente':'Confirmados'}).sort_values(by=['Confirmados'], ascending = False).reset_index()
tabela_rec = saude.loc[(saude['evolucao']==1) & (saude['classificacao_final']==2)].groupby('Área')[['paciente']].count().rename(columns={'paciente':'Recuperados'}).sort_values(by=['Recuperados'], ascending = False).reset_index()
tabela_obt = saude.loc[(saude['evolucao']==2) & (saude['classificacao_final']==2)].groupby('Área')[['paciente']].count().rename(columns={'paciente':'Óbito'}).sort_values(by=['Óbito'], ascending = False).reset_index()
tabela_pcr = saude.loc[(saude['metodo']== 1) & (saude['classificacao_final']==2)].groupby('Área')[['paciente']].count().rename(columns={'paciente':'PCR'}).sort_values(by=['PCR'],  ascending = False).reset_index()
tabela_rap = saude.loc[(~(saude['metodo'].isnull())) & (~(saude['metodo']==1)) & (saude['classificacao_final']==2)].groupby('Área')[['paciente']].count().rename(columns={'paciente':'Teste Rápido/outros'}).sort_values(by=['Teste Rápido/outros'],  ascending = False).reset_index()

In [ ]:
tabela = pd.merge(left=tabela_pcr, right=tabela_rap, on='Área', how='left')
tabela = pd.merge(left=tabela, right=tabela_conf, on='Área',how='left')
tabela = pd.merge(left=tabela, right=tabela_rec, on='Área',how='left')
tabela = pd.merge(left=tabela, right=tabela_obt, on='Área',how='left')
tabela = pd.merge(left=tabela, right=tabela_not, on='Área',how='left')
tabela = tabela.sort_values(by=['Notificados'],  ascending = False)
tabela

In [ ]:
tabela.to_excel(join(default_output,'semanais','TABELA_trabalhadores_saude.xlsx'), index=False)